# Optimize Module

* [1D Optimization](./Optimization_1D.ipynb)
* [ND Optimization](./Optimization_ND.ipynb)
* [Global Optimization](./Optimization_Global.ipynb)
* [1D Roots](./Roots_1D.ipynb)
* [ND Roots](./Roots_ND.ipynb)
* [Curve Fitting](./Curve_Fit.ipynb)


In [2]:
from scipy import optimize

## Common Traits in the Submodule

## Output: `OptimizeResult`

Many functions return an object that can contain more information than simply "This is the minimium".  The information varies between function, method used by the function, and flags given to function, but the way of accessing the data remains the same.  

Let's create one of these data types via minimization to look at it:

In [5]:
f = lambda x : x**2

result=optimize.minimize(f,[2],method="BFGS")

scipy.optimize.optimize.OptimizeResult

You can determine what data types are availible via 

In [6]:
result.keys()

dict_keys(['fun', 'jac', 'hess_inv', 'nfev', 'njev', 'status', 'success', 'message', 'x', 'nit'])

And you can access individual values via:

In [7]:
result.x

array([-1.88846401e-08])

Inspecting the object with `?` or `??` can tell you more about what the individual components actually are.

In Jupyter notebooks, Contextual Help, `Ctrl+I` can also provide this information.

In [10]:
? result

Type:        OptimizeResult
String form:
fun: 3.5662963072207506e-16
           hess_inv: array([[0.5]])
           jac: array([-2.2868119e-08])
           <...> fev: 9
           nit: 2
           njev: 3
           status: 0
           success: True
           x: array([-1.88846401e-08])
Length:      10
File:        ~/anaconda3/lib/python3.7/site-packages/scipy/optimize/optimize.py
Docstring:  
Represents the optimization result.

Attributes
----------
x : ndarray
    The solution of the optimization.
success : bool
    Whether or not the optimizer exited successfully.
status : int
    Termination status of the optimizer. Its value depends on the
    underlying solver. Refer to `message` for details.
message : str
    Description of the cause of the termination.
fun, jac, hess: ndarray
    Values of objective function, its Jacobian and its Hessian (if
    available). The Hessians may be approximations, see the documentation
    of the function in question.
hess_inv : object
    Inver

### `args`

Many routines allow function parameters in a <b>tuple</b> to be passed to the routine via the `args` flag:

In [6]:
f_parameter = lambda x,a : (x-a)**2

optimize.minimize(f_parameter,[0],args=(1,))

      fun: 5.5507662238258444e-17
 hess_inv: array([[0.5]])
      jac: array([4.68181046e-13])
  message: 'Optimization terminated successfully.'
     nfev: 9
      nit: 2
     njev: 3
   status: 0
  success: True
        x: array([0.99999999])

### Methods

The functions in `scipy.optimize` are uniform wrappers that can call to multiple different methods behind the scenes.  For example, `minimize_scalar` can use Brent, Golden, or Bounded methods.  Methods can have different strengths, weaknesses, and pitfalls.  SciPy will automatically choose certain routines given inputted information, but if you know more about the problem, a different routine might be better.

An example of choosing the routine:

In [7]:
f = lambda x : x**2

optimize.minimize(f,[2],method="BFGS")

      fun: 3.5662963072207506e-16
 hess_inv: array([[0.5]])
      jac: array([-2.2868119e-08])
  message: 'Optimization terminated successfully.'
     nfev: 9
      nit: 2
     njev: 3
   status: 0
  success: True
        x: array([-1.88846401e-08])

## Tolerance and Iterations

How much computer time do you want to spend on this problem? How accurate do you need your answer? Is your function really expensive to calculate?

When the two successive values are within the tolerance range of each other or the routine has reached the maximum number of iterations, the routine will exit.  Some functions differentiate between <b>relative tolerance</b> and absolute tolerance</b>.  Relative tolerance scales for the aboslute size of the values.  For example, if two steps are five apart, but each about a trillion, the function can exit.

Whether these quantities exist, and the procedure for setting these quantities varies between functions.  Check individual documentation for details, but here is one example:

In [9]:
optimize.minimize(f,[2],tol=1e-10,options={"maxiter":10})

      fun: 5.55111515902901e-17
 hess_inv: array([[0.5]])
      jac: array([-4.81884952e-17])
  message: 'Optimization terminated successfully.'
     nfev: 12
      nit: 3
     njev: 4
   status: 0
  success: True
        x: array([-7.45058062e-09])

## Getting Help and `scipy.optimize.show_options`

In [3]:
optimize.show_options(solver="minimize_scalar")



brent
=====

Options
-------
maxiter : int
    Maximum number of iterations to perform.
xtol : float
    Relative error in solution `xopt` acceptable for convergence.

Notes
-----
Uses inverse parabolic interpolation when possible to speed up
convergence of golden section method.

bounded

Options
-------
maxiter : int
    Maximum number of iterations to perform.
disp: int, optional
    If non-zero, print messages.
        0 : no message printing.
        1 : non-convergence notification messages only.
        2 : print a message on convergence too.
        3 : print iteration results.
xatol : float
    Absolute error in solution `xopt` acceptable for convergence.

golden

Options
-------
maxiter : int
    Maximum number of iterations to perform.
xtol : float
    Relative error in solution `xopt` acceptable for convergence.
